# Running Scripts in Juypter Lab

Python environment using Anaconda and Jupyter Notebook.

In [ ]:
print("hello drone")

In [ ]:
def sayHello():
    print('Hello')
sayHello()

In [ ]:
def sayHello(drone):
    print('Hello '+ drone)
sayHello("Tello")

## Tello SDK 

The Tello SDK documentation can be found at the following link:

https://dl-cdn.ryzerobotics.com/downloads/tello/0228/Tello+SDK+Readme.pdf

### Download Packet Sender

[Packet Sender](https://packetsender.com/) is a free utility to for sending / receiving of network packets. Support for TCP, UDP, and SSL.


Default Gateway . . . . . . . . . : 192.168.10.1

[socket](https://docs.python.org/3/library/socket.html#module-socket) — Low-level networking interface

This module provides access to the BSD socket interface. It is available on all modern Unix systems, Windows, MacOS, and probably additional platforms.

In [1]:
# This example script demonstrates how to send a basic command over UDP to Tello
# We will use PacketSender to simulate Tello for the purposes of this demonstration.
# PacketSender makes it easy to test code that can then be run when connected to Tello.
# https://packetsender.com/

# This script is part of our course on Tello drone programming
# https://learn.droneblocks.io/p/tello-drone-programming-with-python/

# Import the built-in socket package
import socket

# IP and port of sending computer
# In this case we're sending a UDP packet to PacketSender for demonstration purposes
# Be sure to change this to the IP address of the computer running this Python script

# IP address of the local computer 
# - Mac Terminal command ifconfip
# - Windows Command Prompt ipconfig
simulated_tello_address = ('192.168.10.2', 8889)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Command variable that we'll send
# This "command" value is what lets Tello know that we want to enter command mode
message = "command"

# Send the message to Tello
sock.sendto(message.encode(), simulated_tello_address)

# Print message to screen
print("Message sent!")

Message sent!


## Sending and Receiving UDP Packets

[time](https://docs.python.org/3/library/time.html#module-time) — Time access and conversions

This module provides various time-related functions. For related functionality, see also the datetime and calendar modules.

In [2]:
# This example script demonstrates how to send/receive commands to/from Tello
# This script is part of our course on Tello drone programming
# https://learn.droneblocks.io/p/tello-drone-programming-with-python/

# Import the built-in socket and time modules
import socket
import time

# IP and port of Tello
tello_address = ('192.168.10.1', 8889)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Let's be explicit and bind to a local port on our machine where Tello can send messages
sock.bind(('', 9000))

# Function to send messages to Tello
def send(message):
  try:
    sock.sendto(message.encode(), tello_address)
    print("Sending message: " + message)
  except Exception as e:
    print("Error sending: " + str(e))

# Function that listens for messages from Tello and prints them to the screen
def receive():
  try:
    response, ip_address = sock.recvfrom(128)
    print("Received message: " + response.decode(encoding='utf-8') + " from Tello with IP: " + str(ip_address))
  except Exception as e:
    print("Error receiving: " + str(e))


# Send Tello into command mode
send("command")

# Receive response from Tello
receive()

# Delay 3 seconds before we send the next command
time.sleep(3)

# Ask Tello about battery status
send("battery?")

# Receive battery response from Tello
receive()

# Close the UDP socket
sock.close()

Sending message: command
Error receiving: 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
Sending message: battery?
Received message: ok from Tello with IP: ('192.168.10.1', 8889)


## Tello Box Mission

An Intro to Threading in Python - [Learn more](https://realpython.com/intro-to-python-threading/)

Python threading allows you to have different parts of your program run concurrently and can simplify your design.

**daemon threads**

Daemons are only useful when the main program is running, and it's okay to kill them off once the other non-daemon threads have exited. https://pycozmo.readthedocs.io/en/stable/generated/pycozmo.conn.html#pycozmo.conn.Connection.daemon

In [3]:
# This example script demonstrates how use Python to fly Tello in a box mission
# This script is part of our course on Tello drone programming
# https://learn.droneblocks.io/p/tello-drone-programming-with-python/

# Import the necessary modules
import socket
import threading
import time

# IP and port of Tello
tello_address = ('192.168.10.1', 8889)

# IP and port of local computer
local_address = ('', 9000)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind to the local address and port
sock.bind(local_address)

# Send the message to Tello and allow for a delay in seconds
def send(message, delay):
  # Try to send the message otherwise print the exception
  try:
    sock.sendto(message.encode(), tello_address)
    print("Sending message: " + message)
  except Exception as e:
    print("Error sending: " + str(e))

  # Delay for a user-defined period of time
  time.sleep(delay)

# Receive the message from Tello
def receive():
  # Continuously loop and listen for incoming messages
  while True:
    # Try to receive the message otherwise print the exception
    try:
      response, ip_address = sock.recvfrom(128)
      print("Received message: " + response.decode(encoding='utf-8'))
    except Exception as e:
      # If there's an error close the socket and break out of the loop
      sock.close()
      print("Error receiving: " + str(e))
      break

# Create and start a listening thread that runs in the background
# This utilizes our receive functions and will continuously monitor for incoming messages
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()

# Each leg of the box will be 100 cm. Tello uses cm units by default.
# box_leg_distance = 100
box_leg_distance = 5


# Yaw 90 degrees
yaw_angle = 90

# Yaw clockwise (right)
yaw_direction = "cw"

# Put Tello into command mode / (message, delay)
send("command", 3)

# Send the takeoff command
send("takeoff", 5)

# Fly forward
send("forward " + str(box_leg_distance), 4)

# Yaw right
send("cw " + str(yaw_angle), 3)

# Fly forward
send("forward " + str(box_leg_distance), 4)

# Yaw right
send("cw " + str(yaw_angle), 3)

# Fly forward
send("forward " + str(box_leg_distance), 4)

# Yaw right
send("cw " + str(yaw_angle), 3)

# Fly forward
send("forward " + str(box_leg_distance), 4)

# Yaw right
send("cw " + str(yaw_angle), 3)

# Land
send("land", 5)

# Print message
print("Mission completed successfully!")

# Close the socket
sock.close()

Sending message: command
Received message: ok
Sending message: takeoff
Sending message: forward 5
Received message: out of range
Received message: ok
Sending message: cw 90
Received message: error No valid imu
Sending message: forward 5
Received message: out of range
Sending message: cw 90
Received message: error No valid imu
Sending message: forward 5
Received message: out of range
Sending message: cw 90
Received message: error No valid imu
Sending message: forward 5
Received message: out of range
Sending message: cw 90
Received message: error No valid imu
Sending message: land
Received message: error
Received message: ok
Mission completed successfully!
Error receiving: [WinError 10038] An operation was attempted on something that is not a socket


In [ ]:
# This example script demonstrates how use Python to fly Tello in a box mission
# This script is part of our course on Tello drone programming
# https://learn.droneblocks.io/p/tello-drone-programming-with-python/

# Import the necessary modules
import socket
import threading
import time

# IP and port of Tello
tello_address = ('192.168.10.1', 8889)

# IP and port of local computer
local_address = ('', 9000)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind to the local address and port
sock.bind(local_address)

# Send the message to Tello and allow for a delay in seconds
def send(message, delay):
  # Try to send the message otherwise print the exception
  try:
    sock.sendto(message.encode(), tello_address)
    print("Sending message: " + message)
  except Exception as e:
    print("Error sending: " + str(e))

  # Delay for a user-defined period of time
  time.sleep(delay)

# Receive the message from Tello
def receive():
  # Continuously loop and listen for incoming messages
  while True:
    # Try to receive the message otherwise print the exception
    try:
      response, ip_address = sock.recvfrom(128)
      print("Received message: " + response.decode(encoding='utf-8'))
    except Exception as e:
      # If there's an error close the socket and break out of the loop
      sock.close()
      print("Error receiving: " + str(e))
      break

# Create and start a listening thread that runs in the background
# This utilizes our receive functions and will continuously monitor for incoming messages
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()

# Each leg of the box will be 100 cm. Tello uses cm units by default.
box_leg_distance = 5

# Yaw 90 degrees
yaw_angle = 90

# Yaw clockwise (right)
yaw_direction = "cw"

# Put Tello into command mode / (message, delay)
send("command", 3)

# Send the takeoff command
send("takeoff", 5)

# Fly back
send("back" + str(box_leg_distance), 3)

# Yaw right
send("cw " + str(yaw_angle), 3)

# Fly forward
send("forward " + str(box_leg_distance), 3)

# Land
send("land", 5)

# Print message
print("Mission completed successfully!")

# Close the socket
sock.close()

## Spin and Bounce

In [ ]:
# This example script demonstrates how use Python to create custom flight behaviors with Tello
# This script is part of our course on Tello drone programming
# https://learn.droneblocks.io/p/tello-drone-programming-with-python/

# Import the necessary modules
import socket
import threading
import time

# IP and port of Tello
tello_address = ('192.168.10.1', 8889)

# IP and port of local computer
local_address = ('', 9000)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind to the local address and port
sock.bind(local_address)

# Send the message to Tello and allow for a delay in seconds
def send(message, delay):
  # Try to send the message otherwise print the exception
    try:
        sock.sendto(message.encode(), tello_address)
        print("Sending message: " + message)
    except Exception as e:
        print("Error sending: " + str(e))

    # Delay for a user-defined period of time
    time.sleep(delay)

# Receive the message from Tello
def receive():
  # Continuously loop and listen for incoming messages
    while True:
        # Try to receive the message otherwise print the exception
        try:
            response, ip_address = sock.recvfrom(128)
            print("Received message: " + response.decode(encoding='utf-8'))
        except Exception as e:
            # If there's an error close the socket and break out of the loop
            sock.close()
            print("Error receiving: " + str(e))
            break

# Create and start a listening thread that runs in the background
# This utilizes our receive functions and will continuously monitor for incoming messages
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()

# Initiate command mode and takeoff
def takeoff():
    send("command", 3)
    send("takeoff", 5)

# Land
def land():
    send("land", 5)

# Tello commands respond with an OK when sucessful. This means Tello recognizes
# the command, but the instruction hasn't completed. OK is Tello saying "I got
# the message" but not necessarily saying "I completed the command"
# This means we need to calculate how long the spin will take before we execute the next command.
# Based on our tests a single 360 rotation takes 7 seconds. We'll use this in our spin function to delay
# before the next command. Your rotation time may vary. You can calculate this by
# sending a "cw 360" or "ccw 360" command and measuring the rotation time.

# 7 seconds per rotation
rotationTime = 7

# Spin right or left X number of times
def spin(direction, times):
    # One rotation is 360 degrees
    oneRotation = 360

    # Convert the number of rotations to degrees
    rotations = oneRotation * times

    # Calculate the delay to let the spin function complete
    delay = rotationTime * times

    # Spin right (cw) or left (ccw)
    if (direction == "right"):
        send("cw " + str(rotations), delay)
    elif (direction == "left"):
        send("ccw " + str(rotations), delay)

# Calculate speed per sec
verticalSpeed = 20.0

def bounce(distance, times):

    bounceDelay = distance/verticalSpeed

    for i in range(times):
        send("down " + str(distance), bounceDelay)
        send("up " + str(distance), bounceDelay)

# Takeoff
takeoff()

# Spin right 2 times
spin("right", 2)

# Bounce up and down 60 cm and repeat 3 times
bounce(60, 3)

# Spin left 3 times
spin("left", 3)

# Land
land()

# Close the socket
sock.close()

## Tello Keyboard Commands

This online course will walk you through using Python to program your Tello drone for autonomous flight. As of 4/1/18 the commands supported by Tello are:

 - command
 - takeoff
 - land
 - up xx (fly up a distance from 20 - 500 cm)
 - down xx (fly down a distance from 20 - 500 cm)
 - left xx (fly left a distance from 20 - 500 cm)
 - right xx (fly right a distance from 20 - 500 cm)
 - forward xx (fly forward a distance from 20 - 500 cm)
 - back xx (fly backward a distance from 20 - 500 cm)
 - go x y z speed (fly x y z distance with speed)
 - cw xx (yaw clockwise with angle from 1 - 3600 degrees)
 - ccw xx (yaw counter clockwise with angle from 1 - 3600 degrees)
 - flip x (flip l/r/f/b/bl/br/fl/fr)
	 - l = left
	 - r = right
	 - f = forward
	 - b = backward
	 - bl = backward left
	 - br = backward right
	 - fl = forward left
	 - fr = forward right
- speed x (set speed from 1 - 100 cm/s)
- speed? (get current speed)
- battery? (get current battery percentage)
- time? (get current flight time)



In [ ]:
# Import the necessary modules
import socket
import threading
import time
import sys


In [ ]:
# IP and port of Tello
tello_address = ('192.168.10.1', 8889)

# IP and port of local computer
local_address = ('', 9000)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind to the local address and port
sock.bind(local_address)

# Send the message to Tello and allow for a delay in seconds
def send(message):
  # Try to send the message otherwise print the exception
    try:
        sock.sendto(message.encode(), tello_address)
        print("Sending message: " + message)
    except Exception as e:
        print("Error sending: " + str(e))

# Receive the message from Tello
def receive():
  # Continuously loop and listen for incoming messages
  while True:
    # Try to receive the message otherwise print the exception
    try:
        response, ip_address = sock.recvfrom(128)
        print("Received message: " + response.decode(encoding='utf-8'))
    except Exception as e:
        # If there's an error close the socket and break out of the loop
        sock.close()
        print("Error receiving: " + str(e))
        break
        
# Create and start a listening thread that runs in the background
# This utilizes our receive function and will continuously monitor for incoming messages
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()

# Tell the user what to do
print('Type in a Tello SDK command and press the enter key. Enter "quit" to exit this program.')

# Loop infinitely waiting for commands or until the user types quit or ctrl-c
while True:
  
  try:
    # Read keybord input from the user
    message = input('')
    
    # If user types quit then lets exit and close the socket
    if 'quit' in message:
        print("Program exited sucessfully")
        sock.close()
        break
    
    # Send the command to Tello
    send(message)
    
  # Handle ctrl-c case to quit and close the socket
  except KeyboardInterrupt as e:
    sock.close()
    break